# Working with Catalogs and Schema 

## Course: Getting Started with Databricks for Data Engineering
https://customer-academy.databricks.com/learn/courses/2469/get-started-with-databricks-for-data-engineering?hash=37c86bbf5e6d8b76aa4f54f1fa988c33404292ed&generated_by=1212990

## Contents
1. How to connect to your Catalog and Schema
2. Describe your Catalog, and view files within your Catalog->Schema->Volume
3. Create Delta Tables from CSV

### 1. Connect to your Catalog and Schema

In [0]:
%sql

USE CATALOG fundamentals;

List all schemas within a Catalog

In [0]:
%sql

SELECT schema_name FROM fundamentals.information_schema.schemata

In [0]:
%sql

SELECT current_catalog(), current_schema()

###2. Describe current schema

In [0]:
%sql

DESCRIBE SCHEMA EXTENDED IDENTIFIER('fundamentals.default')

#### Show Tables and Volumes

In [0]:
%sql

SHOW TABLES

In [0]:
%sql

SHOW VOLUMES

#### List files within a Volume

Use LIST statment within spark.sql to list all files within a Volume.
You can get the Volume path by clicking on the eclipses and selecting "Copy Volume Path" option.

In [0]:
spark.sql(f"LIST '/Volumes/fundamentals/default/myfiles'").display()
          
          

###3. Create Delta Table from CSV

View files using file format eg. "csv".
Use SELECT * FROM <FILE FORMAT>.`path to file` in backticks ``

In [0]:
spark.sql(
    f'''
          SELECT *
          FROM csv.`/Volumes/fundamentals/default/myfiles/`
          '''
).display()

This shows the CSV files has a header. Lets read it in using SQL

In [0]:
%sql
SELECT
  *
FROM
  read_files(
    '/Volumes/fundamentals/default/myfiles',
    format => 'csv',
    header => true,
    inferSchema => true
  )

Create delta table named current_employees using CTAS (CREATE TABLE AS) statement 

In [0]:
%sql
USE CATALOG fundamentals;
USE SCHEMA default

In [0]:
%sql
-- Drop table if it already exists to avoid error
DROP TABLE IF EXISTS current_employees;

-- Create Delta table using CSV file
CREATE TABLE current_employees AS
SELECT
  ID,
  FirstName,
  Country,
  Role
FROM
  read_files(
    '/Volumes/fundamentals/default/myfiles',
    format => 'csv',
    header => true,
    inferSchema => true
  );

  -- Display Table
  SELECT * FROM current_employees;

In [0]:
%sql
SELECT current_catalog(), current_schema()

Create Delta Tables using Python (optional method)

In [0]:
# Read the CSV file and create a Spark Dataframe
sdf = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/fundamentals/default/myfiles/")
)
sdf.display()

In [0]:
# Create a Delta Table from the Spark Dataframe
(sdf
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("fundamentals.default.cur_employees_py")
)

Read the delta table using python

In [0]:
spark.read.table("fundamentals.default.cur_employees_py").display()

### View Data from Delta Table

In [0]:
%sql
SELECT *
FROM current_employees;

In [0]:
%sql
DESCRIBE DETAIL current_employees;

In [0]:
%sql
DESCRIBE EXTENDED current_employees;

In [0]:
%sql
DESCRIBE HISTORY current_employees;

### Insert, Update and Delete Records in the Delta Table

In [0]:
%sql
SELECT * FROM current_employees;

In [0]:
%sql

--1. Insert records (2 New employees)
INSERT INTO current_employees
VALUES
  (5555, 'Alex', 'USA', 'Instructor'),
  (6666, 'Sanjay', 'India', 'Instructor');  

--2. Update records (Change Role of Employee 1111)
UPDATE current_employees
SET Role = 'Senior Manager'
WHERE ID = 1111;

--3. Delete records (Remove Employee 3333)
DELETE FROM current_employees
WHERE ID = 3333;

In [0]:
%sql
SELECT * FROM current_employees WHERE Country = 'USA' ORDER BY ID DESC;

In [0]:
%sql
DESCRIBE HISTORY current_employees;

### Time Travel om Delta Tables

In [0]:
%sql
SELECT *
FROM current_employees
VERSION AS OF 2
ORDER BY ID DESC;

### Drop Table

In [0]:
%sql
DROP TABLE IF EXISTS current_employees;
DROP TABLE IF EXISTS cur_employees_py;